# CCNY Calendar Scraper

This notebook will scrape the CCNY Fall 2021 academic calendar.


In [125]:

import requests
from bs4 import BeautifulSoup
import pandas as pd


In [126]:

url = "https://www.ccny.cuny.edu/registrar/fall"
response = requests.get(url)
html_content = response.text


In [127]:
soup = BeautifulSoup(html_content, 'html.parser')
soup.prettify()
table = soup.find_all("table")[0]
rows = soup.find_all("tr")


In [128]:
data = {
    "date": [],
    "dow": [],  
    "text": [],
}


In [129]:
# Loop through table rows and extract data
for tr in rows:
    tds = tr.find_all("td")
    if len(tds) >= 3:
        date_text = tds[0].get_text(strip=True)
        dow_text = tds[1].get_text(strip=True)
        event_text = tds[2].get_text(strip=True)
        
        # Add to data dictionary
        data["date"].append(date_text)
        data["dow"].append(dow_text)
        data["text"].append(event_text)


In [ ]:


print(f"Number of rows collected: {len(data['date'])}")
if len(data['date']) > 0:
    print(f"First few dates: {data['date'][:3]}")
    print(f"First few dow: {data['dow'][:3]}")
    print(f"First few text: {data['text'][:3]}")
else:
    print("No data collected - checking table structure...")
    print(f"Number of rows found: {len(rows)}")
    if len(rows) > 0:
        print("First row cells:")
        first_row = rows[0].find_all("td")
        for i, cell in enumerate(first_row):
            print(f"  Cell {i}: {cell.get_text(strip=True)}")